### 📸 작업 요약: 사진 수동 분류

#### 1. 작업 목적
Jupyter Notebook의 `ipywidgets` 라이브러리를 사용하여 `photo/` 폴더 내의 모든 이미지를 시각적으로 하나씩 확인하며, 4개의 정의된 카테고리(사진보관, 메모확인, 좋아하는그림, 삭제)로 수동 분류 및 이동합니다.

#### 2. 작업 대상
* **원본 경로:** 도커 컨테이너 내부의 `/data/photo` (로컬 `D:\conda\photo`와 연결됨)
* **대상 파일:** `jpg`, `jpeg`, `png`, `heic` 등 이미지 파일
* **분류 폴더:**
    * `/data/01_Keep_Photos`
    * `/data/02_Review_Memos`
    * `/data/03_Favorite_Pics`
    * `/data/04_Delete_Bin`

#### 3. 핵심 기능
* **대화형 GUI:** 노트북 셀에 현재 이미지와 4개의 분류 버튼(보관, 메모, 그림, 삭제)을 표시합니다.
* **HEIC 지원:** `pillow-heif` 라이브러리를 사용해 `.heic` 포맷의 이미지를 실시간 JPEG로 메모리상에서 변환하여, 이미지가 깨지지 않고 정상적으로 보이도록 처리합니다.
* **파일 이동:** 사용자가 버튼을 클릭하면, `shutil.move`를 통해 실제 이미지 파일을 해당 카테고리 폴더로 즉시 이동시킵니다.
* **연도별 스캔:** `folder_start` 변수 값을 통해 특정 연도(`2022`, `2023` 등)로 시작하는 폴더만 선택적으로 스캔하여 작업을 분리수행합니다.

---

### 📊 작업 진행사항 요약

#### 1. 진행 방식
* 스크립트의 `folder_start = '20XX'` 변수 값을 변경하며 연도별로 순차 진행.

#### 2. 현재 상태 (2025-11-02 기준)
* **[완료]** `folder_start = '2022'` (2022년도) 분류 작업 완료. (2025.11.02.)
* **[완료]** `folder_start = '2023'` (2023년도) 분류 작업 완료. (2025.11.02.)
* **[완료]** `folder_start = '2024'` (2024년도) 분류 작업 완료. (2025.11.03.)
* **[대기]** `folder_start = '2025'` (2025년도) 분류 작업 대기 중.

---

### 🚀 향후 작업 진행
* 스크립트 필터 값을 `folder_start = '2024'`로 변경하여 남은 `2024`년도 폴더에 대한 파일명 표준화 작업을 실행할 예정
* 스크립트 필터 값을 `folder_start = '2025'`로 변경하여 남은 `2025`년도 폴더에 대한 파일명 표준화 작업을 실행할 예정


In [7]:
## -y (yes) 플래그를 먼저 쓰거나, 맨 뒤에 씁니다.
## -c conda-forge (채널 지정)를 패키지 이름 앞에 둡니다.
## conda-forge 채널에서 libheif와 libstdcxx-ng를 강제로 설치합니다.
# !conda install -y -c conda-forge libheif libstdcxx-ng

In [8]:
# 필요한 라이브러리 임포트
import os
import shutil
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image

# --- [수정] HEIC 지원을 위한 라이브러리 임포트 ---
import io
import pillow_heif
# --- [수정] Pillow가 HEIC/HEIF 파일을 인식하도록 등록 ---
pillow_heif.register_heif_opener()

print("라이브러리 준비 완료.")

라이브러리 준비 완료.


In [ ]:
# --- 1. 설정 ---
# 원본 사진이 있는 기본 폴더
source_base_dir = "/data/photo"

# (결과물 폴더) 로컬 D:\conda\ 에 생성됩니다.
keep_dir     = "/data/01_Keep_Photos"     # 1. 사진보관
review_dir   = "/data/02_Review_Memos"    # 2. 메모확인
favorite_dir = "/data/03_Favorite_Pics"   # 3. 좋아하는그림
delete_dir   = "/data/04_Delete_Bin"      # 4. 삭제

# --- 2. 폴더 생성 ---
os.makedirs(keep_dir, exist_ok=True)
os.makedirs(review_dir, exist_ok=True)
os.makedirs(favorite_dir, exist_ok=True)
os.makedirs(delete_dir, exist_ok=True)

print(f"대상 폴더 4개 준비 완료:")
print(f" -> {keep_dir} (사진보관)")
print(f" -> {review_dir} (메모확인)")
print(f" -> {favorite_dir} (좋아하는그림)")
print(f" -> {delete_dir} (삭제)")

# --- 3. 모든 이미지 파일 목록 스캔 ---
image_files = []
img_width = 200        # 이미지 표시 영역 : 이미지 높이
valid_extensions = ('.jpg', '.jpeg', '.png', '.heic', '.tif')

# 'photo' 폴더(base_dir)에 처음 진입했을 때,
# 하위 폴더 목록(dirnames)을 '2022'으로 시작하는 폴더로 제한합니다.
# 이렇게 하면 '202212_' 같은 폴더만 탐색하게 됩니다.
folder_start = '2025'

# -----------------------------------------------------------------------

for root, dirs, files in os.walk(source_base_dir):
    # '20XXXX_' 폴더만 탐색
    if root == source_base_dir:
        dirs[:] = [d for d in dirs if d.startswith(folder_start)]

    for file in files:
        if file.lower().endswith(valid_extensions):
            image_files.append(os.path.join(root, file))

# 파일 순서를 거꾸로 (pop을 사용하기 위함)
image_files.reverse()
total_count = len(image_files)

print(f"\n총 {total_count}개의 이미지 파일을 찾았습니다.")

대상 폴더 4개 준비 완료:
 -> /data/01_Keep_Photos (사진보관)
 -> /data/02_Review_Memos (메모확인)
 -> /data/03_Favorite_Pics (좋아하는그림)
 -> /data/04_Delete_Bin (삭제)

총 0개의 이미지 파일을 찾았습니다.


In [10]:
# --- 4. 전역 변수 (현재 보고 있는 파일) ---
current_file_path = None

# --- 5. 위젯 생성 ---
# 버튼 4개s
btn_keep = widgets.Button(description='✅ 1. 사진보관', button_style='success')
btn_review = widgets.Button(description='📝 2. 메모확인', button_style='warning')
btn_favorite = widgets.Button(description='🎨 3. 좋아하는그림', button_style='info') # (신규)
btn_delete = widgets.Button(description='🗑️ 4. 삭제', button_style='danger')
# 버튼 배열
btn_layout = widgets.HBox([btn_keep, btn_review, btn_favorite, btn_delete]) # 버튼을 가로로 배치
img_display = widgets.Image(width=img_width) # 이미지 표시 영역 : 이미지 너비 800px로 고정
info_label = widgets.Label(value="시작하려면 버튼을 클릭하세요.") # 정보 표시 라벨


In [11]:
# --- 6. 핵심 로직: 다음 이미지 표시 ---
# --- [수정] HEIC 변환 로직이 추가된 함수 ---
def show_next_image():
    global current_file_path

    try:
        current_file_path = image_files.pop()
        file_ext_lower = os.path.splitext(current_file_path)[1].lower()

        img_bytes = None

        if file_ext_lower in ['.heic', '.heif']:
            # 1. HEIC 파일인 경우:
            #    Pillow로 열고(register_heif_opener 덕분에 가능)
            #    메모리상에서 JPEG 포맷으로 변환합니다.
            try:
                img = Image.open(current_file_path)

                # 이미지를 저장할 임시 메모리 버퍼 생성
                buffer = io.BytesIO()
                # 'img.save'를 실제 파일이 아닌 메모리 버퍼에 JPEG로 저장
                img.save(buffer, format="JPEG")
                img_bytes = buffer.getvalue() # 버퍼의 바이트 값을 가져옴

            except Exception as e:
                print(f" [오류] HEIC 파일 변환 실패 ({current_file_path}): {e}")
                img_bytes = None # 변환 실패 시
        else:
            # 2. JPG, PNG 등 기존 파일인 경우:
            #    파일의 바이트를 그대로 읽어옵니다.
            try:
                with open(current_file_path, 'rb') as f:
                    img_bytes = f.read()
            except Exception as e:
                 print(f" [오류] 파일 읽기 실패 ({current_file_path}): {e}")
                 img_bytes = None

        # 이미지 위젯의 값을 변환된/읽어온 바이트로 설정
        img_display.value = img_bytes

        # 정보 업데이트
        remaining = len(image_files)
        info_label.value = f"[{remaining+1}/{total_count}] 파일: {current_file_path}"

    except IndexError:
        # 목록이 비었을 때 (작업 완료)
        clear_output() # 모든 위젯 지우기
        print("🎉 모든 사진을 분류했습니다! 수고하셨습니다.")
        current_file_path = None


# --- 7. 버튼 클릭 시 실행될 함수들 ---
def move_file(target_dir):
    if not current_file_path:
        print("작업할 파일이 없습니다.")
        return

    try:
        # 파일 이동
        filename = os.path.basename(current_file_path)
        target_path = os.path.join(target_dir, filename)
        shutil.move(current_file_path, target_path)
        print(f" -> {filename}을(를) {os.path.basename(target_dir)} 폴더로 이동.")

    except Exception as e:
        print(f" [오류] {current_file_path} 이동 실패: {e}")

    # 다음 이미지 표시
    show_next_image()

# 각 버튼에 대한 콜백 함수
def on_keep_clicked(b):
    move_file(keep_dir)

def on_review_clicked(b):
    move_file(review_dir)

def on_favorite_clicked(b):
    move_file(favorite_dir)

def on_delete_clicked(b):
    move_file(delete_dir)


In [12]:
# --- 8. 버튼과 함수 연결 ---
btn_keep.on_click(on_keep_clicked)
btn_review.on_click(on_review_clicked)
btn_favorite.on_click(on_favorite_clicked)
btn_delete.on_click(on_delete_clicked)

# --- 9. UI 표시 및 첫 이미지 로드 ---
print("--- 📸 사진 분류기 시작 ---")
print("첫 사진을 로드합니다...")

# 위젯들을 세로로 배치하여 표시
display(info_label, img_display, btn_layout)

# 첫 이미지 로드
show_next_image()

🎉 모든 사진을 분류했습니다! 수고하셨습니다.
